# Part 1 - Extracting and Saving Data from Yelp API

## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
!pip install yelpapi

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [3]:
# Load API Credentials
with open('/Users/yawagipong/.secret/yelp_api.json', 'r') as f:
    login = json.load(f)

In [4]:
login.items()

dict_items([('client-id', 'DClgQxVVuDAybFmSzqfyKg'), ('api-key', 'Fewiokg7S9jhMiokwKeFI_dV8mII19sqqer4iwaQ5hYHc4MSzMWGTF1GeIKJrC42ByD5YYPqNCWmTYw9XnDq4mVo698xHnBkP17bmkyDdD6bqocsbt6Snvx-w-DXY3Yx')])

In [5]:
login.keys()

dict_keys(['client-id', 'api-key'])

In [6]:
# Instantiate YelpAPI Variable
yelp = YelpAPI(login['api-key'], timeout_s = 5.0)

### Define Search Terms and File Paths

In [7]:
# set our API call parameters and filename before the first call
location = 'Seattle, WA 98122'
term = 'pizza'

In [8]:
## Specify fodler for saving data
FOLDER = 'Data/'
os.makedirs(FOLDER, exist_ok = True)

In [9]:
#splits the location by commas, will just give the first part of dictionary
location.split(',')[0]

'Seattle'

In [10]:
# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = FOLDER + f"{location.split(',')[0]}-{term}.json"

In [11]:
JSON_FILE

'Data/Seattle-pizza.json'

### Check if Json File exists and Create it if it doesn't

In [12]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)

## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    
    ## If JSON_FILE included a folder:
    if len(folder) > 0:
        # create the folder
        os.makedirs(folder, exist_ok = True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE, 'w') as f:
        json.dump([], f)
        
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists")


[i] Data/Seattle-pizza.json already exists


### Load JSON FIle and account for previous results

In [13]:
## Load previous results and use len of results for offset
with open(JSON_FILE, 'r') as f:
    previous_results = json.load(f)

#set offset based on previous results
n_results = len(previous_results)
print(f' - {n_results} previous results found.')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Make the first API call to get the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [ ]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp.search_query(term = term, location = location)

In [ ]:
type(results)

In [ ]:
## How many results total?
len(results)
#3 means there are 3 keys

- Where is the actual data we want to save?

In [ ]:
results.keys()

In [ ]:
results['businesses']

In [ ]:
results['total']
#856 records exist for Seattle pizzas

In [ ]:
pd.DataFrame(results['businesses'])

In [ ]:
## How many did we get the details for?
results_per_page =  len(results['businesses'])
results_per_page

- Calculate how many pages of results needed to cover the total_results

In [ ]:
# Use math.ceil to round up for the total number of pages of results.
import time, math

n_pages = math.ceil((results['total'])/ results_per_page)
n_pages

In [ ]:
for i in tqdm_notebook( range(1,n_pages+1)):
    ## The block of code we want to TRY to run
    try:
        time.sleep(.2)
        ## Read in results in progress file and check the length
        with open(JSON_FILE, 'r') as f:
            previous_results = json.load(f)
        
        ## save number of results for to use as offset
        n_results = len(previous_results)
        
        
        ## use n_results as the OFFSET 
        results = yelp_api.search_query(location = LOCATION,
                                   term = TERM,
                                   offset = n_results + 1)

        ## append new results and save to file
        previous_results.extend(results['businesses'])
        
        with open(JSON_FILE, 'w') as f:
            json.dump(previous_results, f)

            
    ## What to do if we get an error/exception.
    except Exception as e:
        print('[!] ERROR:', e)


## Open the Final JSON File with Pandas

In [ ]:
df = pd.read_json(JSON_FILE)
df.head()

In [ ]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

In [ ]:
## Save it as a compressed csv (to save space)
df.to_csv(csv_file, compression = 'gzip', index = False)

## Bonus: compare filesize with os module's `os.path.getsize`

In [ ]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

## Next Class: Processing the Results and Mapping 